In [2]:
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
hf_token = None
model_path = "/localdisk/ssrivas9/prismatic-vlms/runs/lora-stage-0-after-llava-vqav2/checkpoints/latest-checkpoint.pt"
llm_config = AutoConfig.from_pretrained("lmsys/vicuna-7b-v1.5", token=hf_token)
llm_model = AutoModelForCausalLM.from_pretrained(
        "lmsys/vicuna-7b-v1.5",
        config=llm_config,  # Pass the modified configuration
        token=hf_token,
    )
from peft import (
    LoraConfig,
    PrefixTuningConfig, #Prefix-Tuning
    PromptEncoderConfig, #P-Tuning
    PromptTuningConfig, # Prompt Tuning
    IA3Config,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
lora_r=4
lora_target_modules=["q_proj", "v_proj"]
lora_alpha=16
lora_dropout=0.05
# Add PEFT LoRA on top of this.
llm_model = prepare_model_for_kbit_training(llm_model)
loraconfig = LoraConfig(
    r=lora_r, lora_alpha=lora_alpha, target_modules=lora_target_modules,
    lora_dropout=lora_dropout, bias="none", task_type="CAUSAL_LM"
)
llm_model = get_peft_model(llm_model, loraconfig)

/localdisk/ssrivas9/miniconda3/envs/prism/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.08it/s]
/localdisk/ssrivas9/miniconda3/envs/prism/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/localdisk/ssrivas9/miniconda3/envs/prism/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set 

In [3]:
tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5", model_max_length=2048, token=hf_token)
tokenizer.add_special_tokens({"pad_token": "<PAD>"})

1

In [4]:
llm_model.config.pad_token_id = tokenizer.pad_token_id
llm_model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=64)

Embedding(32064, 4096)

In [5]:
# Load weights from model_path
import torch
state_dicts = (torch.load(model_path))
print(state_dicts.keys())
state_dicts['model']['llm_backbone'].keys()

dict_keys(['model'])


odict_keys(['llm.base_model.model.model.embed_tokens.weight', 'llm.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'llm.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'llm.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'llm.base_model.model.model.layers.0.self_attn.k_proj.weight', 'llm.base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'llm.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'llm.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'llm.base_model.model.model.layers.0.self_attn.o_proj.weight', 'llm.base_model.model.model.layers.0.mlp.gate_proj.weight', 'llm.base_model.model.model.layers.0.mlp.up_proj.weight', 'llm.base_model.model.model.layers.0.mlp.down_proj.weight', 'llm.base_model.model.model.layers.0.input_layernorm.weight', 'llm.base_model.model.model.layers.0.post_attention_layernorm.weight', 'llm.base_model.model.model.layers.1.self

In [6]:
state_dicts['model'].keys()

dict_keys(['projector', 'llm_backbone'])

In [5]:
# Check if the model has the llm prefix in the state dict keys and adjust them if necessary
if any(key.startswith('llm.') for key in state_dicts['model']['llm_backbone'].keys()):
    adjusted_state_dict = {key.replace('llm.', ''): value for key, value in state_dicts['model']['llm_backbone'].items()}
else:
    adjusted_state_dict = state_dicts['model']['llm_backbone']
    # Load the adjusted state dict into the model
    llm_model.load_state_dict(adjusted_state_dict)

# 

In [6]:
llm_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32064, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
        

In [8]:
target_modules

['base_layer',
 'lm_head',
 'k_proj',
 'up_proj',
 'o_proj',
 'default',
 'down_proj',
 'gate_proj']

In [14]:
from transformers import AutoModelForCausalLM, AutoModel
from peft import PeftModelForCausalLM
llm = AutoModelForCausalLM.from_pretrained("/localdisk/ssrivas9/prismatic-vlms/runs/lora-stage-0-after-llava-vqav2-lora-rank-32/checkpoint_llm_only")
#model = PeftModelForCausalLM.from_pretrained(model = llm, model_id="/localdisk/ssrivas9/prismatic-vlms/runs/qlora-stage-0-after-llava-vqav2/checkpoint_llm_only")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.82it/s]


RuntimeError: Error(s) in loading state_dict for LlamaForCausalLM:
	size mismatch for model.embed_tokens.weight: copying a param with shape torch.Size([32064, 4096]) from checkpoint, the shape in current model is torch.Size([32000, 4096]).
	size mismatch for lm_head.weight: copying a param with shape torch.Size([32064, 4096]) from checkpoint, the shape in current model is torch.Size([32000, 4096]).

In [15]:
from transformers import AutoTokenizer
from peft import PeftModel

# Define the path to the directory where the adapter weights are saved
peft_dir = "/localdisk/ssrivas9/prismatic-vlms/runs/qlora-stage-0-after-llava-vqav2/checkpoint_llm_only"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_dir)

# Load the adapter model (PEFT model)
adapter_model = PeftModel.from_pretrained(peft_dir)

# Inspect the weights
adapter_weights = adapter_model.state_dict()

# Print the keys of the adapter weights to see what is stored
print("Adapter Weights Keys:")
for key in adapter_weights.keys():
    print(key)

# If you want to inspect specific weights
print("\nSample Weights:")
for key in adapter_weights.keys():
    print(f"{key}: {adapter_weights[key].shape}")
    break  # remove this line to print all weights


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


TypeError: from_pretrained() missing 1 required positional argument: 'model_id'

In [13]:
llm

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32064, 4096, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm

In [6]:

from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftModelForCausalLM

# Define the path to the directory where the model and tokenizer are saved
peft_dir = Path("/localdisk/ssrivas9/prismatic-vlms/runs/qlora-stage-0-after-llava-vqav2/checkpoint_llm_only")

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(peft_dir)

# # Load the model
# model_config = AutoModelForCausalLM.from_pretrained(peft_dir)
# if isinstance(model_config, PeftModel) or isinstance(model_config, PeftModelForCausalLM):
#     model = model_config.from_pretrained(peft_dir, safe_serialization=False)
# else:
#     raise TypeError("Loaded model is not an instance of supported PEFT model classes.")



Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.73it/s]
/localdisk/ssrivas9/miniconda3/envs/prism/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/localdisk/ssrivas9/miniconda3/envs/prism/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrec

TypeError: Loaded model is not an instance of supported PEFT model classes.

In [ ]:
# Write Script to load a PeftModel from a given path
from peft import load_peft_model
model_dir = ''
model = load_peft_model(model_dir)
model


In [16]:
import torch

# Define the path to the adapter model file
adapter_model_path = "/localdisk/ssrivas9/prismatic-vlms/runs/qlora-stage-0-after-llava-vqav2/checkpoint_llm_only/adapter_model.bin"

# Load the state dictionary from the adapter model file
adapter_state_dict = torch.load(adapter_model_path)

In [19]:
print(adapter_state_dict.keys())

dict_keys(['llm.base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight', 'llm.base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight', 'llm.base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight', 'llm.base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight', 'llm.base_model.model.model.layers.0.mlp.down_proj.lora_A.weight', 'llm.base_model.model.model.layers.0.mlp.down_proj.lora_B.weight', 'llm.base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight', 'llm.base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight', 'llm.base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight', 'llm.base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight', 'llm.base_model.model.model.layers.1.mlp.down_proj.lora_A.weight', 'llm.base_model.model.model.layers.1.mlp.down_proj.lora_B.weight', 'llm.base_model.model.model.layers.2.self_attn.q_proj.lora_A.weight', 'llm.base_model.model.model.layers.2.self_attn.q_proj.lora_B.weight', 'llm.base_model.model

In [12]:

import argparse
import logging
import sys
import json
import os
from pathlib import Path
from typing import List, Optional, Union

from huggingface_hub import hf_hub_download

from prismatic.models.materialize import get_llm_backbone_and_tokenizer, get_vision_backbone_and_transform
from prismatic.models.registry import GLOBAL_REGISTRY, MODEL_REGISTRY
from prismatic.models.vlms import PrismaticVLM
from prismatic.overwatch import initialize_overwatch
from prismatic.models import load

import torch
from lm_eval import evaluator

# Initialize Overwatch =>> Wraps `logging.Logger`
overwatch = initialize_overwatch(__name__)


# === HF Hub Repository ===
# HF_HUB_REPO = "TRI-ML/prismatic-vlms"

SELECTED_NLP_TASKS = ["wsc273","arc_easy","arc_challenge","winogrande","lambada_standard"] #, "webqs"] ["wsc273"]#

# === Available Models ===
def available_models() -> List[str]:
    return list(MODEL_REGISTRY.keys())


def available_model_names() -> List[str]:
    return list(GLOBAL_REGISTRY.items())


def get_model_description(model_id_or_name: str) -> str:
    if model_id_or_name not in GLOBAL_REGISTRY:
        raise ValueError(f"Couldn't find `{model_id_or_name = }; check `prismatic.available_model_names()`")

    # Print Description & Return
    print(json.dumps(description := GLOBAL_REGISTRY[model_id_or_name]["description"], indent=2))

    return description


# === Load Pretrained Model ===


def load_and_write(write_checkpoint_path, model_id_or_path):
    # if model_id_path already contains checkpoint_llm_only folder then skip
    if os.path.exists(os.path.join(write_checkpoint_path, "checkpoint_llm_only")):
        overwatch.info(f"Checkpoint already exists at {write_checkpoint_path}")
        # if checkpoints_base_llm also exists then skip
        if os.path.exists(os.path.join(write_checkpoint_path, "checkpoints_base_llm")):
            overwatch.info(f"Base checkpoint already exists at {write_checkpoint_path}")
            return
        
    vlm = load(model_id_or_path)
    llm_hugface_save_path = os.path.join(
        write_checkpoint_path,
        "checkpoint_llm_only",
    )
    llm_base_path = os.path.join(
        write_checkpoint_path,
        "checkpoints_base_llm",
    )
    overwatch.info(f"Writing LLM & Tokenizer to checkpoint: {llm_hugface_save_path}")
    llm_backbone = vlm.llm_backbone
    llm_backbone.llm.save_pretrained(llm_hugface_save_path)
    llm_backbone.llm.base_model.save_pretrained(llm_base_path)
    llm_backbone.tokenizer.save_pretrained(llm_base_path)
    llm_backbone.tokenizer.save_pretrained(llm_hugface_save_path)
    return llm_hugface_save_path


def nlp_evaluation(llm_checkpoint_path):
    print(f"\n\t NLP Evaluation: {llm_checkpoint_path}")
    task_names = SELECTED_NLP_TASKS
    model_args = f"pretrained={str(llm_checkpoint_path)},trust_remote_code=True"
    print(f"Evaluating on {task_names} with model_args: {model_args}")
    
    results = evaluator.simple_evaluate( 
            model="hf",
            model_args=model_args,
            tasks=task_names,
            device="cuda:0",
            log_samples=True,
            output_path=llm_checkpoint_path
        )
    with open(f"{llm_checkpoint_path}/nlp_evaluation_results.log", "w") as log_file:
        log_file.write(json.dumps(results, indent=4))
    return results


if __name__ == "__main__":
    # parser = argparse.ArgumentParser(description="Process VLM checkpoint for NLP evaluation.")
    # parser.add_argument('--write_path', type=str, required=True, help='Path to write the LLM checkpoint')
    # parser.add_argument('--checkpoint_path', type=str, required=True, help='Path to the VLM checkpoint')
    # args = parser.parse_args()
    
    # main(args.write_path, args.checkpoint_path)
    runs_directory = Path("/localdisk/ssrivas9/prismatic-vlms/runs")
    for checkpoint_dir in runs_directory.iterdir():
        if checkpoint_dir.is_dir():  # ensure it's a directory
            write_path = checkpoint_dir
            print(f"\n Processing: {checkpoint_dir}\n")
            load_and_write(write_path, checkpoint_dir)

OrderedDict([('llm.base_model.model.model.embed_tokens.weight',
              tensor([[ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
                       -6.5565e-06,  8.9407e-07],
                      [ 2.6588e-03, -4.0588e-03,  2.1038e-03,  ..., -9.7656e-03,
                        3.4580e-03, -4.0665e-03],
                      [ 1.0414e-02,  8.3237e-03, -5.6038e-03,  ...,  2.6588e-03,
                       -1.3170e-03, -3.6335e-03],
                      ...,
                      [-9.7424e-03, -4.0932e-02,  1.7835e-03,  ...,  3.2697e-03,
                        2.4879e-03, -4.2195e-03],
                      [-9.8943e-04, -2.1665e-03,  1.9245e-03,  ...,  9.9978e-03,
                       -1.1535e-02,  2.9351e-02],
                      [ 6.5397e-03,  4.3520e-02, -5.6831e-04,  ...,  2.1207e-02,
                       -1.0226e-02, -3.4046e-02]])),
             ('llm.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight',
              tensor([[-0.0060, 